This notebook only shows how we extracted mel spectrogram features by overcoming limited storage.

This notebook is not required to run as our optimized_model_data.csv file in Data folder already contains mel spectrogram which we added using this approach.

In [ ]:
import pandas as pd
import os
import ast

In [ ]:
import numpy as np

In [ ]:
import warnings

In [ ]:
from scipy import stats

In [ ]:
import librosa

In [ ]:
tracks_ids = pd.read_csv("../Data/optimized_model_data.csv",index_col=[0],header=[0,1,2]).index

Getting path to audio files which are names according to track_id.

In [ ]:
def get_audio_path(track_id):

    tid_str = '{:06d}'.format(track_id)
    print(os.path.join("fma_full", tid_str[:3], tid_str + '.mp3'))
    return os.path.join("fma_full", tid_str[:3], tid_str + '.mp3')

In [ ]:
def columns():
    feature_sizes = dict(mel_spec=128)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    return columns.sort_values()

Downloading single track -> Extracting and Saving Features -> Deleting track

In [ ]:
async def getFile(zipe,filepath):
    return zipe.extract(filepath)

In [ ]:
async def compute_features(tid,zipe):

    features = pd.Series(index=columns(), dtype=np.float32, name=tid)

    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        filepath = get_audio_path(tid)
        file = await getFile(zipe,filepath)
        x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.melspectrogram(S=stft)
        feature_stats('mel_spec', f)
        del stft
        os.remove("./"+filepath)

    except Exception as e:
        print('{}: {}'.format(tid, repr(e)))

    return features

In [ ]:
features = pd.DataFrame(index=tracks_ids,
                            columns=columns(), dtype=np.float32)

In [ ]:
async def getMelSpec():

    tids = tracks_ids
    from remotezip import RemoteZip
    with RemoteZip('https://os.unil.cloud.switch.ch/fma/fma_full.zip') as zip:
        for ids in tids:
            f = await compute_features(ids,zip)
            features.loc[f.name] = f

        return features





In [ ]:
mel_spec_features = await getMelSpec()

fma_full/000/000002.mp3
fma_full/000/000003.mp3


Saving extracted mel spectrogram features.

In [ ]:
mel_spec_features.to_csv("../Data/mel_spec_features.csv")